In [1]:
import docx
from utils import *
from fastai.text.all import *
import ipywidgets as widgets
import uuid
from IPython.display import FileLink

In [2]:
out_input = widgets.Output()
out_input

Output()

In [3]:
with out_input:
    print("Идёт загрузка...")

In [4]:
import torch
learn = load_learner('./models/model_50', cpu=not torch.cuda.is_available())

In [5]:
def npa_to_items_test(npa):
    items = []
    for doc in npa:
        for point, point_text in doc.items():
            items.append((point, '\n'.join(point_text)))
    return items

def predict(edition_doc):
    result = []
    npa_items = npa_to_items_test(parse_npa(edition_doc))
    npa_df = pd.DataFrame(npa_items, columns=['point', 'text'])
    test_dl = learn.dls.test_dl(npa_df)
    predictions = learn.get_preds(dl=test_dl)
    threshold = 0.5
    for point, prediction in zip(npa_df['point'].values, predictions[0]):
        factors_3 = []
        factors_4 = []
        for pred, factor in zip(prediction, FACTORS):
            if pred > threshold:
                if factor[0] == '3':
                    factors_3.append(factor[1])
                else:
                    factors_4.append(factor[1])
        if len(factors_3) > 0:
            result.append(('3', factors_3, point))
        if len(factors_4) > 0:
            result.append(('4', factors_4, point))

    return result

In [6]:
uploader = widgets.FileUpload(accept='.docx', multiple=False)

out_input.clear_output()
with out_input:
    print("Загрузите НПА сюда:")
    display(uploader)

In [7]:
out = widgets.Output()
out

Output()

In [8]:
docs_folder = Path() / 'docs'

def on_upload_change(change):
    out.clear_output()
    with out:
        if change.new == 0: return
        doc_id = uuid.uuid4()
        file_path = docs_folder / f'{doc_id}.docx'
        file_result_path = docs_folder / f'{doc_id}_result.docx'
        with open(file_path, 'wb') as file:
            file.write(list(uploader.value.values())[0]["content"])
        uploader._counter = 0
        npa_highlight(file_path, predict(docx.Document(file_path)), file_result_path)
        print('НПА с отмеченными потенциальными коррупциогенными факторами:')
        display(FileLink(file_result_path))

uploader.observe(on_upload_change, names='_counter')